In [ ]:
import pandas as pd
import numpy as np

from itertools import combinations

from scipy.stats import (
    kstest,
    f_oneway,
    ttest_ind,
    kruskal,
    mannwhitneyu,
    chi2_contingency,
    fisher_exact
)

from statsmodels.stats.multitest import multipletests


In [23]:
DATA_PATH = "clustered_sample.csv"   
SEP = ";"                                 
CLUSTER_COL = "cluster"                     
OUTPUT_CSV = "cluster_summary_table.csv"
OUTPUT_XLSX = "cluster_summary_table.xlsx"

# --- LOAD DATA (IMPORTANT PART) ---
df = pd.read_csv(DATA_PATH, sep=",")   # or just pd.read_csv(DATA_PATH)

# Clean possible spaces/BOM from column names
df.columns = (
    df.columns
      .str.strip()            # remove spaces at start/end
      .str.replace('\ufeff', '', regex=False)  # remove BOM if present
)


print("Columns found:", df.columns.tolist())  # debug: see real names


Columns found: ['age', 'gender', 'education', 'marital', 'income', 'AUDIT_total_robust', 'DAST_total_robust', 'IAT_total_robust', 'PGSI_total_robust', 'PCL5_total_robust', 'MSPSS_total_robust', 'SWLS_total_robust', 'WHO5_total_robust', 'cluster']


In [10]:

# 1) DEMOGRAPHIC / CATEGORICAL VARIABLES (ORDERED)
DEMO_CATEGORICAL = ["gender", "education", "marital", "income_cat"]

# 2) OTHER NUMERIC VARIABLES
OTHER_NUMERIC = [
    "AUDIT_total_robust",
    "DAST_total_robust",
    "IAT_total_robust",
    "PGSI_total_robust",
    "PCL5_total_robust",
    "MSPSS_total_robust",
    "SWLS_total_robust",
    "WHO5_total_robust",
]


# Mapping from numeric codes to labels for categorical vars
CATEGORY_LABELS = {
    "gender": {
        0: "Male",
        1: "Female",
        2: "Non-binary",
        3: "Prefer not to say",
    },
    "education": {
        5: "Elementary school",
        8: "Middle school",
        13: "High School",
        18: "Bachelor's Degree",
        22: "Master's Degree",
        25: "Doctoral Degree",
    },
    "marital": {
        0: "Single",
        1: "Married",
        2: "Divorced",
        3: "Widowed",
        4: "Separated",
        5: "Prefer not to say",
    }
}


# Pretty labels for left column
DISPLAY_NAMES = {
    "gender": "Gender",
    "education": "Education level",
    "marital": "Marital status",
    "income_cat": "Income bracket",
    "AUDIT_total_robust": "Alcohol use (AUDIT)",
    "DAST_total_robust": "Drug use (DAST)",
    "IAT_total_robust": "Internet Addiction (IAT)",
    "PGSI_total_robust": "Gambling (PGSI)",
    "PCL5_total_robust": "Trauma (PCL-5)",
    "MSPSS_total_robust": "Social Support (MSPSS)",
    "SWLS_total_robust": "Life Satisfaction (SWLS)",
    "WHO5_total_robust": "Well-being (WHO-5)",
}



In [11]:
def format_p_value(p):
    if pd.isna(p):
        return ""
    if p < 0.001:
        return "<0.001"
    return f"{p:.3f}"

def summarize_numeric(df, var, cluster_col):
    """For numeric variables: median (Q1; Q3) + Kruskal-Wallis"""
    result = {}
    groups = {}

    for cl, sub in df.groupby(cluster_col):
        values = sub[var].dropna().astype(float)
        groups[cl] = values
        if len(values) == 0:
            result[cl] = ""
        else:
            median = np.median(values)
            q1 = np.percentile(values, 25)
            q3 = np.percentile(values, 75)
            result[cl] = f"{median:.1f} ({q1:.1f}; {q3:.1f})"

    valid = [g for g in groups.values() if len(g) > 0]
    if len(valid) >= 2:
        _, p = stats.kruskal(*valid)
    else:
        p = np.nan

    result["p"] = format_p_value(p)
    return result

def summarize_categorical(df, var, cluster_col):
    """For categorical variables: mode + % + chi-square test"""
    result = {}

    if var in CATEGORY_LABELS:
        mapping = CATEGORY_LABELS[var]
        series_all = df[var].map(mapping)
    else:
        series_all = df[var].astype(str)

    for cl, sub in df.groupby(cluster_col):
        series = series_all.loc[sub.index].dropna()
        if len(series) == 0:
            result[cl] = ""
        else:
            counts = series.value_counts()
            mode_label = counts.idxmax()
            pct = 100 * counts.max() / len(series)
            result[cl] = f"{mode_label} ({pct:.0f}%)"

    contingency = pd.crosstab(df[var], df[cluster_col])
    if contingency.shape[0] >= 2 and contingency.shape[1] >= 2:
        _, p, _, _ = stats.chi2_contingency(contingency)
    else:
        p = np.nan

    result["p"] = format_p_value(p)
    return result

In [12]:
df.columns = df.columns.str.strip().str.replace("\ufeff", "", regex=False)

# Convert cluster to string for consistent grouping
df[CLUSTER_COL] = df[CLUSTER_COL].astype(str)


# Convert income to categorical (Option B)
df["income_cat"] = pd.cut(
    df["income"],
    bins=[0, 20000, 40000, 60000, np.inf],
    labels=["<20k", "20k–40k", "40k–60k", ">60k"],
    right=False
)

clusters = sorted(df[CLUSTER_COL].unique())
cluster_counts = {cl: (df[CLUSTER_COL] == cl).sum() for cl in clusters}
cluster_col_names = [f"Cluster {cl} (n={cluster_counts[cl]})" for cl in clusters]

rows = []
# 1) DEMOGRAPHIC CATEGORICAL
for var in DEMO_CATEGORICAL:
    summary = summarize_categorical(df, var, CLUSTER_COL)
    row = {"Variable": DISPLAY_NAMES.get(var, var)}
    for cl, col_name in zip(clusters, cluster_col_names):
        row[col_name] = summary.get(cl, "")
    row["P value"] = summary["p"]
    rows.append(row)

# 2) OTHER NUMERIC VARIABLES
for var in OTHER_NUMERIC:
    summary = summarize_numeric(df, var, CLUSTER_COL)
    row = {"Variable": DISPLAY_NAMES.get(var, var)}
    for cl, col_name in zip(clusters, cluster_col_names):
        row[col_name] = summary.get(cl, "")
    row["P value"] = summary["p"]
    rows.append(row)


In [13]:
# Create final table
table_df = pd.DataFrame(rows, columns=["Variable"] + cluster_col_names + ["P value"])

# Save to Excel
with pd.ExcelWriter(OUTPUT_XLSX, engine="openpyxl") as writer:
    table_df.to_excel(writer, index=False, sheet_name="Cluster Summary")

print("Excel file created:", OUTPUT_XLSX)
print("Cluster sizes:", cluster_counts)

Excel file created: cluster_summary_table.xlsx
Cluster sizes: {'0': np.int64(56), '1': np.int64(54), '2': np.int64(55), '3': np.int64(56)}


In [ ]:

"""
- Read cluster labels and assign each cluster a significance symbol (* # $ % ...).
- These symbols will be used later to mark “significant difference from Cx”.
"""

# ========= CONFIG =========
FILE = "cluster_sample.csv"      # <-- put your xlsx filename here
CLUSTER_COL = "cluster"         # <-- name of your cluster column

# tell the script which variables are nominal and which are ordinal
NOMINAL_VARS = ["gender", "marital"]      # edit if names differ
ORDINAL_VARS = ["education", "income"]    # numeric codes with order

ALPHA = 0.05

# Ensure cluster is treated as category-like
clusters = sorted(df[CLUSTER_COL].dropna().unique())

# Map each cluster to a symbol (first 4)
symbol_list = ["*", "#", "$", "%", "&", "@"]  # just in case >4 clusters
cluster_to_symbol = {c: symbol_list[i] for i, c in enumerate(clusters)}
print("Cluster → Symbol mapping:", cluster_to_symbol)



Cluster → Symbol mapping: {np.int64(0): '*', np.int64(1): '#', np.int64(2): '$', np.int64(3): '%'}


In [ ]:
"""
Helper functions for Bonferroni correction and marker-table creation.

Functions:
- bonferroni_correct(p_values):
    Applies Bonferroni correction to a list of p-values.
    Formula: p_corrected = min(p * number_of_tests, 1)

- empty_marker_table():
    Creates an empty table with clusters as columns and no rows yet.
    This table will later be filled with symbols (*, #, $, ...) indicating
    which clusters differ significantly from which.
"""

def bonferroni_correct(p_values):
    """Simple Bonferroni correction: p * m, clipped to 1."""
    m = len(p_values)
    return [min(p * m, 1.0) for p in p_values]

def empty_marker_table():
    """Create an empty marker table: rows=variables, cols=clusters."""
    return pd.DataFrame("", index=[], columns=clusters)

# we’ll fill two marker tables:
markers_numeric = empty_marker_table()
markers_nominal = empty_marker_table()


In [ ]:
"""
Statistical testing for numeric and ordinal variables.

Process:
1. Automatically extract all numeric variables.
2. Remove variables that should not be treated as numeric
   (cluster column, nominal variables).
3. For each variable:
    - Split values by cluster.
    - Run Kruskal–Wallis test (non-parametric general test across >2 groups).
    - If significant:
         * perform pairwise Mann–Whitney tests between cluster pairs.
         * apply Bonferroni correction.
         * write symbols indicating which clusters differ.
"""

# ---- find numeric variables automatically ----
numeric_all = df.select_dtypes(include=[np.number]).columns.tolist()

# remove cluster and nominal (they are coded as numbers but not numeric)
for c in [CLUSTER_COL] + NOMINAL_VARS:
    if c in numeric_all:
        numeric_all.remove(c)

# numeric_all now contains: ordinal + continuous variables
# we will treat all of them with Kruskal + Mann–Whitney (non-parametric)

for var in numeric_all:
    data = df[[CLUSTER_COL, var]].dropna()
    if data.empty:
        continue

    # groups per cluster
    groups = [data[data[CLUSTER_COL] == c][var].values for c in clusters]

    # --- General test: Kruskal–Wallis ---
    if len(groups) >= 2:
        stat, p_general = kruskal(*groups)
    else:
        continue

    if p_general >= ALPHA:
        # no overall difference → no marks
        continue

    # --- Pairwise Mann–Whitney with Bonferroni ---
    pair_names = []
    pair_ps = []

    for c1, c2 in combinations(clusters, 2):
        g1 = data[data[CLUSTER_COL] == c1][var].values
        g2 = data[data[CLUSTER_COL] == c2][var].values
        if len(g1) == 0 or len(g2) == 0:
            continue

        _, p_pair = mannwhitneyu(g1, g2, alternative="two-sided")
        pair_names.append((c1, c2))
        pair_ps.append(p_pair)

    if not pair_ps:
        continue

    pair_ps_corr = bonferroni_correct(pair_ps)

    # --- Build markers for this variable ---
    # start with empty marks for each cluster
    marks_for_var = {c: "" for c in clusters}

    for (c1, c2), p_corr in zip(pair_names, pair_ps_corr):
        if p_corr < ALPHA:
            # c1 is different from c2 → in the cell for c1 we add symbol of c2
            marks_for_var[c1] += cluster_to_symbol[c2]
            # and symmetric: c2 is different from c1 → add symbol of c1
            marks_for_var[c2] += cluster_to_symbol[c1]

    # write row into markers_numeric
    markers_numeric.loc[var] = pd.Series(marks_for_var)


In [ ]:
"""
Statistical testing for nominal (categorical) variables.

Process:
1. Construct a contingency table: categories × clusters.
2. Clean table by removing rows/columns with zero total.
3. Perform Chi-square test to detect overall association.
4. If significant:
       * perform pairwise cluster comparisons using Chi-square
         or Fisher's exact test (for 2×2 tables or low expected counts).
       * apply Bonferroni correction.
       * assign symbols indicating which clusters differ.
"""

results_nom = []

for var in NOMINAL_VARS:
    data = df[[CLUSTER_COL, var]].dropna()
    if data.empty:
        continue

    contingency = pd.crosstab(data[var], data[CLUSTER_COL])

    # 🧹 Remove rows/cols with zero total – they break chi2
    contingency = contingency.loc[contingency.sum(axis=1) > 0, :]
    contingency = contingency.loc[:, contingency.sum(axis=0) > 0]

    # if <2 clusters or <2 categories left, we can’t test → skip
    if contingency.shape[0] < 2 or contingency.shape[1] < 2:
        continue

    try:
        chi2, p_general, dof, expected = chi2_contingency(contingency)
    except ValueError:
        # still problematic (rare) → skip this variable
        continue

    if p_general >= ALPHA:
        # no overall association → no marks
        continue

    # small expected counts? → use Fisher for 2x2 pairwise
    use_fisher = (expected < 5).any()

    pair_names = []
    pair_ps = []

    for c1, c2 in combinations(contingency.columns, 2):
        table2 = contingency[[c1, c2]]

        if use_fisher and table2.shape == (2, 2):
            _, p_pair = fisher_exact(table2.values)
            pair_test = "Fisher"
        else:
            try:
                _, p_pair, _, _ = chi2_contingency(table2)
                pair_test = "Chi-square"
            except ValueError:
                # if still invalid, skip this pair
                continue

        pair_names.append((c1, c2))
        pair_ps.append(p_pair)

    if not pair_ps:
        continue

    pair_ps_corr = bonferroni_correct(pair_ps)

    # build symbols for this nominal variable
    marks_for_var = {c: "" for c in clusters}

    for (c1, c2), p_corr in zip(pair_names, pair_ps_corr):
        if p_corr < ALPHA:
            # c1 differs from c2 → add symbol of c2 in row c1
            if c2 in cluster_to_symbol:
                marks_for_var[c1] += cluster_to_symbol[c2]
            # and symmetric
            if c1 in cluster_to_symbol:
                marks_for_var[c2] += cluster_to_symbol[c1]

    markers_nominal.loc[var] = pd.Series(marks_for_var)


In [31]:
markers_numeric.to_excel("markers_numeric_ordinal.xlsx")
markers_nominal.to_excel("markers_nominal.xlsx")

print("Saved:")
print(" - markers_numeric_ordinal.xlsx")
print(" - markers_nominal.xlsx")


Saved:
 - markers_numeric_ordinal.xlsx
 - markers_nominal.xlsx


Results reflect:


Cluster 0: the Overwhelmed Student Looking for Support

Mostly females, often married, with lower income and elementary–to–high-school education. They show **moderate** alcohol, drug, and internet-addiction scores, along with mid-level well-being and trauma exposure. Mia represents adults who face daily pressures and rely on emotional support systems, navigating life with limited economic resources.

Cluster 1: the Social Drinker on the Edge

Predominantly male, married, with lower educational background and mid-range income. They show **the lowest** internet-addiction scores but **elevated alcohol-use and gambling-risk levels**, along with **lower life satisfaction** and emotional well-being. Marko reflects adults who cope socially through substance use while struggling with emotional stability.

Cluster 2: the Always-Online Friend

Mostly single individuals with higher education (often Master’s degree) and **the highest income**. They show **very high internet-use scores**, moderate gambling and alcohol risk, and **strong well-being, social support, and life satisfaction**. Alex represents a digitally engaged, high-achieving adult who maintains good emotional health despite heavy online activity.

Cluster 3: the Caring Professional with High Well-Being

Predominantly women, often with middle-school to higher education and low-to-mid income. They show moderate substance-use scores and high trauma exposure, but also **the strongest emotional well-being and social support** among all clusters. Sara represents resilient, socially connected adults who thrive emotionally despite facing life stressors.
